# SSMR: generate SSM model from trajectory data

In [1]:
import numpy as np
from scipy.io import loadmat
from copy import deepcopy

In [2]:
%matplotlib qt
import matplotlib.pyplot as plt

In [3]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

In [4]:
np.set_printoptions(linewidth=300)

In [5]:
use_data = "john" # "john"

In [6]:
# import os
# import pickle

# diamond_dir = "../../../soft-robot-control/examples/diamond"
# decayData_dir = os.path.join(diamond_dir, "dataCollection/")

# OUTPUT_NODE = 1354
# t_in = 1.2
# t_out = 3.0

# oData = []

# with open(os.path.join(diamond_dir, 'rest.pkl'), 'rb') as file:
#     q_rest = np.array(pickle.load(file)['rest'])

# print(q_rest)

# decayData_files = sorted([f for f in os.listdir(decayData_dir) if '.pkl' in f]) # [0:1]
# plt.close('all')

# for i, traj in enumerate(decayData_files):
#     with open(os.path.join(decayData_dir, traj), 'rb') as file:
#         data = pickle.load(file)

#     t = np.array(data['t'])
#     q_tip = (np.array(data['q'])[:, 3*OUTPUT_NODE:3*OUTPUT_NODE+3] - q_rest[3*OUTPUT_NODE:3*OUTPUT_NODE+3]).T
#     q_dot_tip = np.gradient(q_tip, axis=1)

#     # remove time until t_in
#     ind = (t_in <= t) & (t <= t_out)
#     t = t[ind] - t_in
#     q_tip = q_tip[:, ind]
#     q_dot_tip = q_dot_tip[:, ind]

#     oData_traj = np.vstack([q_tip, q_dot_tip])

#     oData.append([t, oData_traj])

## Import and inspect data
See mat file in URL

In [7]:
dataDecayObs = loadmat("../dataDecayObs.mat") # , simplify_cells=True)


Data = {}
if use_data == "john":
    Data['oData'] = dataDecayObs['oData'].tolist()
    # Data['yData'] = dataDecayObs['yData'].tolist()
    # Xsnapshots = dataDecayObs['Xsnapshots']
    for i in range(len(Data['oData'])):
        Data['oData'][i][0] = Data['oData'][i][0].squeeze()
        # Data['yData'][i][0] = Data['yData'][i][0].squeeze()
else: # use my data
    Data['oData'] = oData

H = np.array(dataDecayObs['H'])
V = np.array(dataDecayObs['V'])
M = np.array(dataDecayObs['M'])

# oData = dataDecayObs['oData']
# yData = dataDecayObs['yData']

rDOF = 3
oDOF = 3
nTRAJ = len(Data['oData'])
# nDOF = len(Xsnapshots)//2

print("nTRAJ:", nTRAJ)
print(H.shape, V.shape, M.shape)

nTRAJ: 52
(4884, 4) (9768, 6) (4884, 4884)


In [8]:
# print(V[:nDOF, :rDOF].shape)
# print(Data['oData'][0][1][:3, :].shape)
# test_y = V[:nDOF, :rDOF].T @ Data['oData'][0][1][:3, :]
# print(test_y.shape)
# print(np.allclose(test_y, Data['yData'][0][1]))

### Plot trajectories in an observable of interest

In [9]:
outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot evolution of x, y and z in time, separately in 3 subplots
plot.traj_xyz(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot trajectories in 3D [x, x_dot, z] space / NB: x_dot = v_x
plot.traj_3D(Data,
             xyz_idx=[('oData', 0), ('oData', 0+oDOF), ('oData', 2)],
             xyz_names=[r'$x$', r'$\dot{x}$', r'$z$'])

Compute reduced coordinates using delay embedding on the available oData

In [10]:
Data['yData'] = deepcopy(Data['oData'])
for i in range(nTRAJ):
    delay_embedded_traj = utils.delayEmbedding(Data['oData'][i][1], up_to_delay=4)
    Data['yData'][i][1] = delay_embedded_traj[:, :-4]
    Data['yData'][i][0] = Data['yData'][i][0][4:]

In [11]:
# Data['oData'][0][1][:, :10]

In [12]:
# print(Data['yData'][0][0].shape, Data['yData'][0][1].shape)
# Data['yData'][0][1][:, :5]

In [13]:
t_interval = [0.14, 2.6]
Data['oDataTrunc'] = utils.slice_trajectories(Data['oData'], t_interval)
Data['yDataTrunc'] = utils.slice_trajectories(Data['yData'], t_interval)

In [14]:
# print(Data['yDataTrunc'][0][1].shape)
# Data['yDataTrunc'][0][1][:, :5]

## Obtain reduced-order coordinates

### Perform PCA on delay-embedded coordinates to obtain reduced coordinates

In [15]:
# Perform SVD on displacement field
# if use_data == "mine":
from scipy.sparse.linalg import svds

Xsnapshots = np.hstack([yDataTrunc[1] for yDataTrunc in Data['yDataTrunc']])
# subsample dt=0.01
Xsnapshots = Xsnapshots[:, ::10]

# print(Xsnapshots.shape)
# print(Xsnapshots[:, 0:10])

v, s, u = svds(Xsnapshots, k=6, which="LM")
ind = np.argsort(s)[::-1]
s = s[ind]
v = v[:, ind]
# v, s, _ = np.linalg.svd(Xsnapshots, compute_uv=True)
# print(s)
# print(v)
l2vals = s**2

In [16]:
v

array([[ 1.01956581e-01, -9.21880467e-02,  4.27299700e-01,  2.13248182e-01, -1.67898979e-01,  5.72452508e-01],
       [-5.14517774e-02, -4.38339866e-01, -8.20352879e-02, -1.15774514e-01, -6.07938090e-01, -1.35280273e-01],
       [ 4.35592362e-01, -3.05694648e-02, -1.11320517e-01,  5.84687146e-01, -5.87505828e-02, -2.34401638e-01],
       [ 1.01611316e-01, -9.17293617e-02,  4.26754308e-01,  1.05955097e-01, -8.32876082e-02,  2.83397760e-01],
       [-5.09870215e-02, -4.37703315e-01, -8.18118697e-02, -5.76689907e-02, -3.01049375e-01, -6.71567748e-02],
       [ 4.34221307e-01, -3.01021336e-02, -1.10247266e-01,  2.88597027e-01, -2.93481965e-02, -1.15966258e-01],
       [ 1.01239240e-01, -9.12439610e-02,  4.26078570e-01, -5.22574245e-04,  6.74482547e-04, -3.62501323e-03],
       [-5.05078877e-02, -4.36935324e-01, -8.15658025e-02, -4.65884478e-05,  3.66973277e-03,  5.60147181e-04],
       [ 4.32758271e-01, -2.96354305e-02, -1.09162807e-01, -5.02678210e-03, -5.13895911e-05,  1.51310995e-03],
 

In [17]:
# Plot variance description: we expect three modes to capture almost all variance.
# Note we assume data centered around the origin, which is the fixed point of our system.
plt.close('all')
plot.pca_modes(l2vals, up_to_mode=9)

### Project delay-embedded data onto dominant modes

In [18]:
from copy import deepcopy
Data['etaDataTrunc'] = deepcopy(Data['yDataTrunc'])
SSMDim = 6
Vde = v[:, :SSMDim]
for i in range(nTRAJ):
    Data['etaDataTrunc'][i][1] = Vde.T @ Data['yDataTrunc'][i][1]

In [19]:
Data['etaDataTrunc'][0][1][:, :5]

array([[25.74353323, 26.01548912, 26.2751917 , 26.52269308, 26.75804898],
       [44.48464414, 45.45355914, 46.40124784, 47.32755067, 48.23231777],
       [ 4.11636981,  4.28676953,  4.45619188,  4.62458428,  4.79189481],
       [-0.54500073, -0.52583341, -0.5066859 , -0.48756475, -0.46847626],
       [-1.5101942 , -1.48439876, -1.45829789, -1.43190461, -1.40523194],
       [-0.24783287, -0.24749892, -0.24707771, -0.24657   , -0.24597661]])

In [20]:
plt.close('all')
# plot first three reduced coordinates
plot.traj_xyz(Data,
              xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 1), ('etaDataTrunc', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'])

### Train and test data (train/test split)

In [21]:
indTest = [0, 2, 4, 6, 8, 10, 12, 14]
indTrain = [i for i in range(nTRAJ) if i not in indTest]

## SSMLearn

Start Matlab engine and install/run SSMLearn

In [22]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.cd("../..", nargout=0)
eng.install(nargout=0)
eng.cd("ROM/")

SSMLearn: Data-driven Reduced Order Models for Nonlinear Dynamical Systems
Maintained by Mattia Cenedese (mattiac@ethz.ch) and Joar Axs (jgoeransson@ethz.ch)
License: GNUv3.0
> In path (line 109)
In addpath (line 86)
In install (line 20)


'/home/jonas/Projects/stanford/SSMR-for-control'

Bring data in a format that is accepted by the (slightly modified) version of SSMLearn (requires a specific cell array structure for input data)

In [23]:
# make data ready for Matlab
yDataTruncTrain_matlab = [[], []]
etaDataTruncTrain_matlab = [[], []]
for i in indTrain:
    yDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['yDataTrunc'][i][0].tolist()))
    yDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['yDataTrunc'][i][1].tolist()))
    etaDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['etaDataTrunc'][i][0].tolist()))
    etaDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['etaDataTrunc'][i][1].tolist()))


### Learn geometry of the SSM

Find parametrization of SSM using SSMLearn

In [24]:
SSMDim = 2*rDOF
obsDim = 2*oDOF
SSMOrder = 3

IMInfo = eng.IMGeometry(yDataTruncTrain_matlab, SSMDim, SSMOrder,
                        'reducedCoordinates', etaDataTruncTrain_matlab)
# IMInfoInv = eng.IMGeometry(yDataTruncTrain_matlab, obsDim, SSMOrder,
#                            'reducedCoordinates', oDataTruncTrain_matlab)


Data seems to have been fed from Python!


In [25]:
np.array(IMInfo['parametrization']['H'])

array([[ 1.01962690e-01, -9.21911874e-02,  4.27302257e-01, ...,  4.09101164e-05,  1.18111170e-04,  5.08472336e-06],
       [-5.14511687e-02, -4.38344048e-01, -8.20357237e-02, ...,  4.59872975e-05, -2.70030467e-04, -4.77358599e-05],
       [ 4.35551032e-01, -3.05669371e-02, -1.11295195e-01, ..., -7.55447305e-05, -2.76211344e-05, -1.03964633e-04],
       ...,
       [ 1.00421994e-01, -9.01977590e-02,  4.24343482e-01, ...,  3.82420130e-05,  1.14311338e-04,  5.65031151e-06],
       [-4.95069574e-02, -4.35014138e-01, -8.10070339e-02, ...,  4.44543349e-05, -2.67704996e-04, -4.72408097e-05],
       [ 4.29518860e-01, -2.87015513e-02, -1.06936054e-01, ..., -7.69554606e-05, -2.05630754e-05, -1.05206974e-04]])

### Learn dynamics on the SSM (reduced dynamics)

Find parametrization of reduced dynamics using SSMLearn

In [26]:
ROMOrder = 3
RDInfo = eng.IMDynamicsFlow(etaDataTruncTrain_matlab, 'R_PolyOrd', ROMOrder, 'style', 'default')

Data seems to have been fed from Python!
Estimation of the reduced dynamics... 
 Done. 


In [27]:
np.array(RDInfo['reducedDynamics']['coefficients'])

array([[-3.57556301e+00,  1.05104875e+00,  1.91750494e+00, -7.00281632e+02,  2.89090674e+01,  9.12651121e+01,  1.61445958e-05,  6.23093123e-08, -5.44146863e-06, -5.61675794e-05,  1.39556888e-04, -2.99683097e-04, -1.40229187e-07,  1.04653519e-06, -3.17041143e-04, -2.08483271e-04, -4.29764136e-05,
        -9.01717098e-08, -1.56027807e-04,  3.52283347e-04,  2.13908148e-05,  2.08613666e-03,  2.94487445e-03, -3.37538704e-03, -8.36837172e-04, -6.18240489e-03,  7.64772492e-04,  6.73024810e-08,  1.09547951e-07,  3.73182196e-10, -4.58934045e-06, -1.56122993e-05, -5.61625275e-06, -6.45350860e-08,
        -1.48039001e-07,  2.11327087e-05, -6.23310903e-06,  3.55877007e-05, -5.43204206e-08,  8.54705455e-06, -3.90749998e-05,  4.28403101e-07, -1.15896229e-04,  3.51695771e-04,  6.98767688e-04,  7.10300821e-04, -5.34009507e-04, -1.51819901e-04, -5.15829150e-08,  8.16491424e-09,  7.51549379e-06,
        -2.44781280e-06,  6.44595051e-06,  6.58614133e-08, -8.44284942e-06, -4.95477809e-06, -1.98033174e-05,

Stop Matlab engine -- not needed anymore

In [28]:
eng.quit()

## Analyze the obtained mappings of SSM geometry and reduced dynamics

### SSM geometry (parametrization)

In [29]:
yRec = {}
errorGeo = {}
meanErrorGeo = {}

#### Train error

In [30]:
yRec['Train'] = utils.lift_trajectories(IMInfo, [Data['etaDataTrunc'][i] for i in indTrain])
errorGeo['Train'] = utils.compute_trajectory_errors(yRec['Train'], [Data['yDataTrunc'][i] for i in indTrain])[0] * 100

meanErrorGeo['Train'] = np.mean(errorGeo['Train'])
print(f"Average parametrization train error: {meanErrorGeo['Train']:.4e}")

Average parametrization train error: 1.1909e-04


#### Test error

In [31]:
yRec['Test'] = utils.lift_trajectories(IMInfo, [Data['etaDataTrunc'][i] for i in indTest])
errorGeo['Test'] = utils.compute_trajectory_errors(yRec['Test'], [Data['yDataTrunc'][i] for i in indTest])[0] * 100

meanErrorGeo['Test'] = np.mean(errorGeo['Test'])
print(f"Average parametrization test error: {meanErrorGeo['Test']:.4e}")

Average parametrization test error: 1.0340e-04


Plot comparison of SSM-predicted vs. actual test trajectories

In [32]:
print(Data['yData'][0][1].shape)
print(yRec['Test'][0][1].shape)

(15, 2996)
(15, 2460)


In [33]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('Test', 0), ('Test', 1), ('Test', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

### Reduced dynamics

In [34]:
etaRec = {}
etaRec['RD'] = utils.advectRD(RDInfo, Data['etaDataTrunc'])[0]
meanErrorDyn = {}

normedTrajDist = utils.compute_trajectory_errors(etaRec['RD'], Data['etaDataTrunc'])[0]
meanErrorDyn['Train'] = np.mean(normedTrajDist[indTrain]) * 100
meanErrorDyn['Test'] = np.mean(normedTrajDist[indTest]) * 100

print(f"Average dynamics train error: {meanErrorDyn['Train']:.4f}")
print(f"Average dynamics test error: {meanErrorDyn['Test']:.4f}")

Average dynamics train error: 0.8390
Average dynamics test error: 0.6147


In [35]:
plot_all = False
if plot_all:
    vPlot = range(nTRAJ)
else:
    vPlot = indTest

plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 1), ('etaDataTrunc', 2)],
                    xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
                    traj_idx=vPlot,
                    show=False)
plot.traj_xyz(etaRec,
              xyz_idx=[('RD', 0), ('RD', 1), ('RD', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
              traj_idx=vPlot,
              axs=axs, ls=':', color='darkblue')


### Global error

In [36]:
errorGlo = {}
meanErrorGlo = {}

#### Train error

In [37]:
yRec['RDTrain'] = utils.lift_trajectories(IMInfo, [etaRec['RD'][i] for i in indTrain])
errorGlo['Train'] = utils.compute_trajectory_errors(yRec['RDTrain'], [Data['yDataTrunc'][i] for i in indTrain])[0] * 100

meanErrorGlo['Train'] = np.mean(errorGlo['Train'])
print(f"Average global train error: {meanErrorGlo['Train']:.4f}")

Average global train error: 0.8390


#### Test error

In [38]:
yRec['RDTest'] = utils.lift_trajectories(IMInfo, [etaRec['RD'][i] for i in indTest])
errorGlo['Test'] = utils.compute_trajectory_errors(yRec['RDTest'], [Data['yDataTrunc'][i] for i in indTest])[0] * 100

meanErrorGlo['Test'] = np.mean(errorGlo['Test'])
print(f"Average global test error: {meanErrorGlo['Test']:.4f}")

Average global test error: 0.6147


In [39]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('RDTest', 0), ('RDTest', 1), ('RDTest', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

## Control

### Setup model for control

In [40]:
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d 

In [77]:
Rauton = utils.Rauton(RDInfo)
Vauton = lambda x: IMInfo['parametrization']['H'] @ utils.multivariate_polynomial(x, IMInfo['parametrization']['polynomialOrder'])

### Learn control matrix $B$

Use figure8 trajectory as the training data

In [75]:
# Load the controls and actual trajectories for figure8
u_train = np.loadtxt("../../system_data/u_big.csv", delimiter=',').T
z_train = np.loadtxt("../../system_data/z_big.csv", delimiter=',').T
z_eq_train = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
print(z_eq_train)

# tip position, normalized using resting position
z_shift_train = (z_train[-3:, :].T - z_eq_train).T

# original spacing of inputs
T = 10.01
dt = 0.01
N = int(T / dt)
t_train = np.linspace(0, T, N+1)

[-7.07686434  1.2132194  97.04053866]


Use randomly sampled (step) inputs as the training data

In [43]:
# diamond_train_full = loadmat("../diamond_train_full.mat", simplify_cells=True)
# z_train, u_train, _ = diamond_train_full['y'].T, diamond_train_full['u'].T, diamond_train_full['t']
# print(z_train.shape)

# # tip position, normalized using resting position
# z_shift_train = (z_train[-3:, :].T - z_train[-3:, -1]).T
# dt = 0.01
# t_train = np.arange(z_train.shape[1]) * dt

Plot the training data

In [76]:
plt.close('all')
plot.traj_xyz_txyz(t=t_train,
                   x=z_shift_train[0, :], y=z_shift_train[1, :], z=z_shift_train[2, :])
plot.inputs(t_train, u_train)

Do delay embedding on the tip position (and optionally, on the inputs). Then, compute reduced coordinates and gradient as predicted by reduced dynamics, as well as the numerical (true) gradient

In [65]:
# delay-embed tip position (as well as inputs?)
y_train = utils.delayEmbedding(z_shift_train)
# u = utils.delayEmbedding(u, embed_coords=[0, 1, 2, 3], up_to_delay=4)
print(f"y.shape: {y_train.shape}, u.shape: {u_train.shape}")
# reduced coordinates
print(t_train[:5])
x_train = Vde.T @ y_train
print(x_train[0, :5])
dxDt = np.gradient(x_train, dt, axis=1)
dxDt_ROM = Rauton(x_train)

# optionally, throw away some of the training data
# keep_indices = np.s_[4000:-100]
keep_indices = np.s_[30:-20]
t_train_trunc = t_train[keep_indices]
u_train_trunc = u_train[:, keep_indices]
x_train_trunc = x_train[:, keep_indices]
y_train_trunc = y_train[:, keep_indices]
dxDt = dxDt[:, keep_indices]
dxDt_ROM = dxDt_ROM[:, keep_indices]

print(t_train.shape)

y.shape: (15, 1002), u.shape: (4, 1002)
[0.   0.01 0.02 0.03 0.04]
[1.26993504 2.14181405 3.22675802 4.47443942 5.84008615]
(1002,)


In [46]:
plt.close('all')
ax = plot.traj_3D_xyz(y_train_trunc[outdofs[0], :],
                      y_train_trunc[outdofs[1], :],
                      y_train_trunc[outdofs[2], :], show=True)

Plot gradients (numerical differentiation and reduced dynamics)

In [78]:
plt.close('all')
scales_of_coordinates = np.ones(dxDt.shape[0])
# scales_of_coordinats = np.amax(np.abs(dxDt), axis=1).reshape(-1, 1)
# scales_of_coordinats = np.mean(dxDt, axis=1)
scales_of_coordinates = scales_of_coordinates.reshape(-1, 1)
print(scales_of_coordinates.shape)
dxDt = dxDt / scales_of_coordinates
dxDt_ROM = dxDt_ROM / scales_of_coordinates
print(dxDt.shape)
plot.reduced_coordinates_gradient(t_train_trunc, [dxDt, dxDt_ROM], labels=["true numerical", "predicted autonomous"], how="all")

(6, 1)
(6, 952)


Learn $B$ matrix

In [79]:
from sklearn.linear_model import Ridge, Lasso

polyUorder = 1

# X = np.vstack([U, Xbar])
U = utils.multivariate_polynomial(u_train_trunc, order=polyUorder)
print(U.shape)

ridgeModel = Ridge(alpha=0, fit_intercept=False)
ridgeModel.fit(U.T, (dxDt - dxDt_ROM).T)
B_learn = ridgeModel.coef_
print(B_learn.shape)

(4, 952)
(6, 4)


Frobenius norm of $B_{learn}$ to estimate how much influence inputs have on dynamics

In [83]:
print(f"Frobenius norm of B_learn: {np.linalg.norm(B_learn, ord='fro'):.4f}")

Frobenius norm of B_learn: 0.0033


Plot gradients again, but this time, using the learned influence of inputs for the prediction

In [82]:
plt.close('all')
plot_reduced_coords = [3, 4, 5]
dxDt_ROM_with_B = Rauton(x_train_trunc) / scales_of_coordinates + B_learn @ utils.multivariate_polynomial(u_train_trunc, order=polyUorder)
plot.reduced_coordinates_gradient(t_train_trunc, [dxDt[plot_reduced_coords, :], dxDt_ROM[plot_reduced_coords, :], dxDt_ROM_with_B[plot_reduced_coords, :]], labels=["true numerical", "predicted autonomous", "predicted with inputs"], how="all")

In [51]:
# # home-brewed ridge regression
# alpha = 0
# X, y = U.T, (dXbarDt - dXbarDt_ROM).T
# B_learn = np.linalg.solve(X.T @ X + alpha * np.eye(X.shape[0]), X.T @ y)

### Integrate model with inputs (big amplitude) -- and using learned influence of control

Try to predict the training trajectory using reduced dynamics

In [52]:
# Load the controls and actual trajectories for figure8
u_test = np.loadtxt("../../system_data/u_big.csv", delimiter=',').T
z_test = np.loadtxt("../../system_data/z_big.csv", delimiter=',').T
z_eq_test = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
print(z_test.shape)
# tip position, normalized using resting position
z_shift_test = (z_test[-3:, :].T - z_eq_test).T
# delay-embed tip position (as well as inputs?)
y_test = utils.delayEmbedding(z_shift_test)
# reduced coordinates
x_test = Vde.T @ y_test

# original spacing of inputs
T = 10.01
dt = 0.01
N = int(T / dt)
t_test = np.linspace(0, T, N+1)

slice_traj = np.s_[30:-20]
t_test = t_test[slice_traj]
u_test = u_test[:, slice_traj]
x_test = x_test[:, slice_traj]
z_test = z_test[:, slice_traj]

(6, 1002)


In [53]:
x_test[:, 0]

uInterpFun = interp1d(t_test, u_test, axis=1, fill_value="extrapolate")
uFun = lambda t: uInterpFun(t).reshape(-1, 1)
# print(t_test.shape, dxDt.shape)
# dxDtInterpFun = interp1d(t_test, dxDt_ROM_with_B, axis=1, fill_value="extrapolate")
def R(t, y):
    return Rauton(np.atleast_2d(y)) + B_learn @ utils.multivariate_polynomial(uFun(t), order=polyUorder)
    # return dxDtInterpFun(t)

# solve IVP of reduced dynamics using open-loop figure8 inputs
sol = solve_ivp(R,
                t_span=[t_test[0], t_test[-1]],
                t_eval=t_test,
                y0=x_test[:, 0], # np.zeros(2*rDOF),
                method='RK45',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
# resulting (predicted) open-loop trajectory in reduced coordinates
xTraj = sol.y
yTraj = Vauton(xTraj)
zTraj = (yTraj[:3, :].T + z_eq_test).T

plt.close('all')
ax = plot.traj_3D_xyz(zTraj[outdofs[0], :],
                      zTraj[outdofs[1], :],
                      zTraj[outdofs[2], :], show=False)
ax = plot.traj_3D_xyz(z_test[3, :],
                      z_test[4, :],
                      z_test[5, :], color="tab:orange", ax=ax, show=False)
ax.legend(["Predicted trajectory", "Actual trajectory"])
plt.show()


In [54]:
RMSE = np.sum(np.sqrt(np.mean((zTraj[0:3, :] - z_test[-3:])**2, axis=0))) / zTraj.shape[1]
print(f"RMSE = {RMSE:.4f}")

RMSE = 2.4964


## Save SSM model

Continuous model

In [55]:
SSM_model = {'model': {}, 'params': {}}

SSM_model['model']['w_coeff'] = IMInfo['parametrization']['H']
SSM_model['model']['v_coeff'] = Vde # IMInfo['chart']['H']
SSM_model['model']['r_coeff'] = RDInfo['reducedDynamics']['coefficients'] 
SSM_model['model']['B'] = B_learn
SSM_model['model']['V'] = Vde

SSM_model['params']['SSM_order'] = SSMOrder
SSM_model['params']['ROM_order'] = ROMOrder
SSM_model['params']['state_dim'] = 2 * rDOF
SSM_model['params']['input_dim'] = H.shape[1]
SSM_model['params']['output_dim'] = 2 * oDOF
# SSM_model['params']['delays'] = 
# SSM_model['params']['obs_dim'] = 

In [56]:
import pickle
# save dictionary to person_data.pkl file
with open('../../../soft-robot-control/examples/hardware/SSMmodels/SSM_model.pkl', 'wb') as f:
    pickle.dump(SSM_model, f)